(https://github.com/tiagoantao/bioinf-python/blob/master/notebooks)

- 1) PLINK
- 2) Genepop
- 3) Bio.PopGene 으로 dataset 탐색하기
- 4) F-statistics (Fst score, Fixation index) 계산
- 5) PCA (Principal Components Analysis)
- 6) Population structure with Admixture

# Population genetics 란,

각 인종에서의 DNA 서열 data를 기반으로 Natural selection, drift, mutation, and migration 등을 연구함. (특히 allele frequency정보 이용)

- 이 chanper에서 사용할 data
    - : [Human HapMap project data](http://www.sanger.ac.uk/resources/downloads/human/hapmap3.html)
        - 햅맵 프로젝트는 common SNP (AF > 1%) 의 catalog를 밝히고 haplotype map을 작성함
        
population|full name
---|---
ASW|African ancestry in Southwest USA
CEU|Utah residents with Northern and Western European ancestry from the CEPH collection
CHB|Han Chinese in Beijing, China
CHD|Chinese in Metropolitan Denver, Colorado
GIH|Gujarati Indians in Houston, Texas
JPT|Japanese in Tokyo, Japan
LWK|Luhya in Webuye, Kenya
MEX|Mexican ancestry in Los Angeles, California
MKK|Maasai in Kinyawa, Kenya
TSI|Toscani in Italia
YRI|Yoruba in Ibadan, Nigeria
        
haplotype? 한 염색체 상에서 서로 통계적으로 연관된(LD) SNP들의 그룹

![HamMap](http://hapmap.ncbi.nlm.nih.gov/images/haplotype_page/hapmapfig2.jpg)

- LD?
![haplotype](http://143.248.31.90:8889/notebooks/note/biopython/figs/haplotype.png)
![haplotype2](http://143.248.31.90:8889/notebooks/note/biopython/figs/haplotype2.png)
![haplotype3](http://143.248.31.90:8889/notebooks/note/biopython/figs/haplotype3.png)

- 1) [PLINK](http://pngu.mgh.harvard.edu/~purcell/plink/) ([version 1.9](https://www.cog-genomics.org/plink2)) -speed and memory improvement
    - analysis of genotype/phenotype data (GWAS study) 분석을 위해서 주로 사용되어 왔음.

In [ ]:
!wget http://hapmap.ncbi.nlm.nih.gov/downloads/genotypes/hapmap3/plink_format/draft_2/hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2
!wget http://hapmap.ncbi.nlm.nih.gov/downloads/genotypes/hapmap3/plink_format/draft_2/hapmap3_r2_b36_fwd.consensus.qc.poly.ped.bz2

In [ ]:
!bunzip2 hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2
!bunzip2 hapmap3_r2_b36_fwd.consensus.qc.poly.ped.bz2

## 1-1) PLINK file format
(1) **map** file (hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2)

chromosome/rsID/Genetic distance (morgans)/Base-pair position (bp units)

In [7]:
! head -3 hapmap3_r2_b36_fwd.consensus.qc.poly.map

1	rs10458597	0	554484
1	rs2185539	0	556738
1	rs11240767	0	718814


(2) **ped** file (hapmap3_r2_b36_fwd.consensus.qc.poly.ped.bz2) 

Family ID/Individual ID/Paternal ID/Maternal ID/Sex (1=male; 2=female; other=unknown)/Phenotype/Genotypes

In [8]:
! head -3 hapmap3_r2_b36_fwd.consensus.qc.poly.ped |cut -f1-10

2427	NA19919	NA19908	NA19909	1	-9	C C	C C	C C	A A
2431	NA19916	0	0	1	-9	C C	C C	T C	A A
2424	NA19835	0	0	2	-9	C C	C C	C C	A A


## 1-2) PLINK 예시1. subsampling
--**recode** automatically generate an updated version (with all filters applied)

--**file** .ped + .map filename prefix

--**out** prefix for output files

--**thin** [p] Randomly remove variants, retaining each with prob. [p]
    - p % 만 남기고 random하게 variant를 선택함
    - seed option
        
--**geno** {val} Exclude variants with missing call frequencies greater than a threshold (default 0.1)
    - default 0.1 : genotyping rate이 90% 보다 작은 경우 filter함

In [13]:
! plink --recode --file hapmap3_r2_b36_fwd.consensus.qc.poly --out hapmap10 --thin 0.1 --geno 0.1
#--thin: 1296959 variants removed (143657 remaining).
! plink --recode --file hapmap3_r2_b36_fwd.consensus.qc.poly --out hapmap1 --thin 0.01 --geno 0.1
#--thin: 1426074 variants removed (14542 remaining).

PLINK v1.90b3.29 64-bit (24 Dec 2015)      https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hapmap10.log.
Options in effect:
  --file hapmap3_r2_b36_fwd.consensus.qc.poly
  --geno 0.1
  --noweb
  --out hapmap10
  --recode
  --thin 0.1

Note: --noweb has no effect since no web check is implemented yet.
17938 MB RAM detected; reserving 8969 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (1440616 variants, 1184 people).
--file: hapmap10-temporary.bed + hapmap10-temporary.bim +
hapmap10-temporary.fam written.
1440616 variants loaded from .bim file.
1184 people (589 males, 595 females) loaded from .fam.
--thin: 1296959 variants removed (143657 remaining).
Using 1 thread (no multithreaded calculations invoked.
Before main variant filters, 988 founders and 196 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%

## 1-2) PLINK 예시2. autosomal chromosome 이외 filter하기

i) map_file을 열어서 chrom>22 인 rsID만 exclude_file에 저장한다.

In [14]:
def get_non_auto_SNPs(map_file, exclude_file):
    f = open(map_file)
    w = open(exclude_file, 'w')
    for l in f:
        toks = l.rstrip().split('\t')
        chrom = int(toks[0])
        rs = toks[1]
        if chrom > 22: 
            w.write('%s\n' % rs)
    w.close()
get_non_auto_SNPs('hapmap10.map', 'exclude10.txt')
get_non_auto_SNPs('hapmap1.map',  'exclude1.txt')

ii) --exclude 옵션을 사용하여 제외한다. (참고: --extract)

In [20]:
!plink --recode --file hapmap10 --out hapmap10_auto --exclude exclude10.txt
!plink --recode --file hapmap1 --out hapmap1_auto --exclude exclude1.txt

PLINK v1.90b3.29 64-bit (24 Dec 2015)      https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hapmap10_auto.log.
Options in effect:
  --exclude exclude10.txt
  --file hapmap10
  --out hapmap10_auto
  --recode

17938 MB RAM detected; reserving 8969 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (143657 variants, 1184 people).
--file: hapmap10_auto-temporary.bed + hapmap10_auto-temporary.bim +
hapmap10_auto-temporary.fam written.
143657 variants loaded from .bim file.
1184 people (589 males, 595 females) loaded from .fam.
--exclude: 138648 variants remaining.
Using 1 thread (no multithreaded calculations invoked.
Before main variant filters, 988 founders and 196 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%2

## 1-2) PLINK 예시3. sample filter하기

  --filter-cases       : Include only cases in the current analysis.
  
  --filter-controls    : Include only controls.
  
  --filter-males       : Include only males.
  
  --filter-females     : Include only females.
  
  --filter-founders    : Include only founders.
  
  --filter-nonfounders : Include only nonfounders

For most population genetic analysis, it's **important** to assure that there is **little relatedness among sampled individuals**

In [4]:
!plink --file hapmap10_auto --filter-founders --recode --out hapmap10_auto_noofs
#196 people removed due to founder status (--filter-founders).
#unrelated individuals needed for most population genetic analysis

PLINK v1.90b3.29 64-bit (24 Dec 2015)      https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hapmap10_auto_noofs.log.
Options in effect:
  --file hapmap10_auto
  --filter-founders
  --out hapmap10_auto_noofs
  --recode

17938 MB RAM detected; reserving 8969 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (138648 variants, 1184 people).
--file: hapmap10_auto_noofs-temporary.bed + hapmap10_auto_noofs-temporary.bim +
hapmap10_auto_noofs-temporary.fam written.
138648 variants loaded from .bim file.
1184 people (589 males, 595 females) loaded from .fam.
196 people removed due to founder status (--filter-founders).
Using 1 thread (no multithreaded calculations invoked.
Before main variant filters, 988 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%

## 1-2) PLINK 예시4. LD로 filter

- LD로 filter하는 과정은 PCA나 Admixture algorithm 전에 요구된다.

i) --indep-pairwise [window size]<kb> [step size (variant ct)] [r^2 threshold]

    a) consider a window of 50 SNPs
    b) calculate LD between each pair of SNPs in the window
    c) remove one of a pair of SNPs if the LD is greater than 0.1
    d) shift the window 10 SNPs forward and repeat the procedure.

In [24]:
!plink --file hapmap10_auto_noofs --indep-pairwise 50 10 0.1 --out keep
# out1 -keep.prune.in
# out2 -keep.prune.out

PLINK v1.90b3.29 64-bit (24 Dec 2015)      https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to keep.log.
Options in effect:
  --file hapmap10_auto_noofs
  --indep-pairwise 50 10 0.1
  --out keep

17938 MB RAM detected; reserving 8969 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (138648 variants, 988 people).
--file: keep-temporary.bed + keep-temporary.bim + keep-temporary.fam written.
138648 variants loaded from .bim file.
988 people (488 males, 500 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked.
Before main variant filters, 988 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%4

ii) --extract 사용하여 keep.prune.in에 속하는 SNP만 추출한다.(참고: --exclude)

In [23]:
!plink --file hapmap10_auto_noofs --extract keep.prune.in --recode --out hapmap10_auto_noofs_ld

PLINK v1.90b3.29 64-bit (24 Dec 2015)      https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hapmap10_auto_noofs_ld.log.
Options in effect:
  --extract keep.prune.in
  --file hapmap10_auto_noofs
  --out hapmap10_auto_noofs_ld
  --recode

17938 MB RAM detected; reserving 8969 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (138648 variants, 988 people).
--file: hapmap10_auto_noofs_ld-temporary.bed +
hapmap10_auto_noofs_ld-temporary.bim + hapmap10_auto_noofs_ld-temporary.fam
written.
138648 variants loaded from .bim file.
988 people (488 males, 500 females) loaded from .fam.
--extract: 55165 variants remaining.
Using 1 thread (no multithreaded calculations invoked.
Before main variant filters, 988 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%

## 1-2) PLINK 예시5. 다른 format으로 저장

--recode <01 | 12>
    - recode01 : minor allele - 0, major allele -1.
    - recode12 : minor allele - 1, major allele -2.
    (나중에 PCA분석에 사용할 것임)
--make-bed
    - Create a new binary fileset.(ped,map --> bed,fam,bim format으로 저장)
        - plink.fam      ( first six columns of mydata.ped ) 
        - plink.bed      ( binary file, genotype information )
        - plink.bim      ( extended MAP file: two extra cols = allele names)

In [1]:
!plink --file hapmap10_auto_noofs_ld --recode12 tab --out hapmap10_auto_noofs_ld_12
!plink --make-bed --file hapmap10_auto_noofs_ld --out hapmap10_auto_noofs_ld

PLINK v1.90b3.29 64-bit (24 Dec 2015)      https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Note: --recode12 flag deprecated.  Use 'recode 12 ...'.
Logging to hapmap10_auto_noofs_ld_12.log.
Options in effect:
  --file hapmap10_auto_noofs_ld
  --out hapmap10_auto_noofs_ld_12
  --recode 12 tab

17938 MB RAM detected; reserving 8969 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (55165 variants, 988 people).
--file: hapmap10_auto_noofs_ld_12-temporary.bed +
hapmap10_auto_noofs_ld_12-temporary.bim +
hapmap10_auto_noofs_ld_12-temporary.fam written.
55165 variants loaded from .bim file.
988 people (488 males, 500 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked.
Before main variant filters, 988 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%

In [7]:
! head -3 hapmap10_auto_noofs_ld_12.ped |cut -f1-10

2431	NA19916	0	0	1	-9	2 2	2 2	2 2	1 2
2424	NA19835	0	0	2	-9	2 2	2 2	2 2	2 2
2469	NA20282	0	0	2	-9	2 2	2 2	2 2	2 2


In [8]:
! head -3 hapmap10_auto_noofs_ld.map 

1	rs3748593	0	870253
1	rs3748595	0	877423
1	rs1126573	0	939233


In [10]:
! head -3 hapmap10_auto_noofs_ld.fam

2431 NA19916 0 0 1 -9
2424 NA19835 0 0 2 -9
2469 NA20282 0 0 2 -9


In [15]:
! head -3 hapmap10_auto_noofs_ld.bim

1	rs3748593	0	870253	A	C
1	rs3748595	0	877423	A	C
1	rs1126573	0	939233	T	C


In [16]:
! head -1 hapmap10_auto_noofs_ld.bed

l��>�����������������������������������������������������������������������������������������������������������������������������������˿����������������������������������?����>���>�����γ�����������������������������������������������������ﾾ�������������������������������������������������������������������������������~/�������������������������������������������������?���������������������������������������������������������������������������������������:����������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������_�������������������������������������������������������������������/ʺ?����������������������������������������/��������￻���������������������������������������������������������������*���ϫ*��/��������������//<����#����?�.�������뿊�:�����<�������������������������


## 1-2) PLINK 예시6. 특정 chromosome만 저장

In [17]:
!plink --recode --file hapmap10_auto_noofs --chr 2 --out hapmap10_auto_noofs_2

PLINK v1.90b3.29 64-bit (24 Dec 2015)      https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hapmap10_auto_noofs_2.log.
Options in effect:
  --chr 2
  --file hapmap10_auto_noofs
  --out hapmap10_auto_noofs_2
  --recode

17938 MB RAM detected; reserving 8969 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (11605 variants, 988 people).
--file: hapmap10_auto_noofs_2-temporary.bed +
hapmap10_auto_noofs_2-temporary.bim + hapmap10_auto_noofs_2-temporary.fam
written.
11605 variants loaded from .bim file.
988 people (488 males, 500 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked.
Before main variant filters, 988 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%

In [18]:
! head -3 hapmap10_auto_noofs_2.map

2	rs13386087	0	24503
2	rs10173732	0	31404
2	rs300757	0	62515
